In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab5.ipynb")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
rng_seed = 454

---

<h1><center>SDSE Lab 5 <br><br> Scikit-learn, logistic regression, feature selection, and regularization</center></h1>

---

In this lab we will build a model for diagnosing breast cancer from various measurements of a tumor. To do this we will use [scikit-learn](https://scikit-learn.org/stable/), which is a package for performing a host of machine learning tasks. We will learn about scikit-learn's train-test data splitter, its standard scaler, pipelines, cross-validation, and LASSO regularization. 

The lab has 12 parts across four sections.

**Prelminaries**

1. Load the data
2. Extract test data
3. Normalize the training data

**Simple logistic regression**

4. Most correlated feature
5. Train simple logistic regression with normalized and unnormalized inputs
6. Create a scikit-learn pipeline
7. Evaluate the models with cross-validation
8. Evaluate the models with test data

**Regularization**

9. LASSO regularized logistic regression
10. Choose the best model
11. Significant features
12. Evaluate the final model with test data


---

<h1><center><font color='purple'> Preliminaries</font><br></center></h1>


# 1. Load the data

This is a [classic dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)) that originates from the University of Wisconsin and is included in the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php), as well as in scikit-learn's collection of [toy datasets](https://scikit-learn.org/stable/datasets/toy_dataset.html). It can be loaded with the [load_breast_cancer](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) method. Passing `as_frame=True` prompts the loader to return a pandas DataFrame. 

This dataset encodes a benign tumor as a 1 and a malignant tumor as a 0. We flip these tags so that the encoding agrees with our common notion of "positive" and "negative" diagnoses. 

In [ ]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer(as_frame=True).frame
data['target'] = 1-data['target']

Use `data.info()` to display a summary of the dataset. 

In [ ]:
data.info()

# 2. Extract test data

The first step is to set aside a portion of the data for final testing. Use scikit-learn's [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to create the testing and training datasets. 

Note: `train_test_split` takes these arguments:
1. The input samples: Use `data.iloc` to select all rows and all but the last column. 
2. The target (output) samples: The last column of `data` (named "target")
3. `test_size` is the portion of the dataset reserved for testing. You should set this to 20% (0.2).
4. Pass `random_state=rng_seed` to fix the random seed and ensure reproducibility of the results. 

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(...,      # TODO
                                                ...,      # TODO
                                                test_size=...,       # TODO
                                                random_state=rng_seed )

In [ ]:
grader.check("q2")

# 3. Normalize the training data

Next we will normalize the data, as we have done before, by subtracting its mean and dividing each column by its standard deviation. This is not strictly necessary for un-regularized logistic regression from a theoretical viewpoint. The algorithm is the same with normalized and un-normalized data. However it can have beneficial effects on the numerical robustness of the optimization solver. 

We use scikit-learn's [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to perform the normalization on the training input data (`Xtrain`). Then we put the result into a new pandas DataFrame.

**Hint** How can you get the index and column names of a pandas DataFrame?

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(Xtrain)
Xtrain_norm = pd.DataFrame(X, index=..., columns=...)    # TODO

In [ ]:
grader.check("q3")


---

<h1><center><font color='purple'> Simple logistic regression</font><br></center></h1>


# 4. Most correlated feature

Our first model will be a simple logistic regression model based on the single feature that best correlates with the output. Find this feature and save its name (i.e. its header value) to `best_single_feature`. 

Note: The tests for this part are hidden.

In [ ]:
...         # TODO
best_single_feature = ...   # TODO

In [ ]:
grader.check("q4")

# 5. Train simple logistic regression

Next we train the simple logistic regression model for the feature that was selected in the previous part. We will use scikit-learn's implementation of [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for this purpose. 

1. Pass `random_state=rng_seed` into the LogisticRegression constructor to ensure repeatability of the results. 
2. Call the [`fit`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.fit) function of the model object, passing in the training data. The model inputs correspond to the single best feature already identified.
3. Extract the trained model coefficients. The intercept term $\hat\theta_0$ is contained in the `intercept_[0]` attribute of the model. The remaining coefficients $\hat\theta_1$ through $\hat\theta_P$ (in this case just $\hat\theta_1$) are in `coef_[0,:]`.

This has been done for you with the original (un-normalized) input data. Repeat the exercise with the normalized data. 

In [ ]:
from sklearn.linear_model import LogisticRegression

model_nonorm = LogisticRegression(random_state=rng_seed)
model_nonorm.fit(Xtrain[[best_single_feature]],ytrain) 
print(model_nonorm.intercept_[0], model_nonorm.coef_[0,:])

model_norm = ...

In [ ]:
grader.check("q5")

# 6. Create a scikit-learn pipeline

Scikit-learn provides a *pipeline* class that collects all of the preprocessing, feature transformation, and modeling components into a single object with `fit` and `predict` methods. You can  read the documentation on [pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) to learn more. 

Each component in the pipeline is identified with a string name. The following code creates a pipeline with a `StandardScaler` tagged as `scaler`, followed by a logistic regression model tagged as `logreg`.

``` python
pipe = Pipeline([('scaler', StandardScaler()), 
                 ('logreg', LogisticRegression(random_state=rng_seed)) ])
```

Create this pipeline and train it on the `best_single_feature` of the un-normalized dataset (`Xtrain`,`ytrain`) using the `fit` method. 

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline(...)
pipeline.fit(...) 

In [ ]:
grader.check("q6")

# 7. Evaluate the models with cross-validation

Accuracy is an important performance metric for classification models. It is computed as the ratio of correct predictions to the total number of predictions. Hence it approximates the probability that the prediction is correct. 

K-fold cross-validation is an evaluation technique that provides a robust estimate of model performance (e.g. accuracy) without the need for test data. It does this by splitting the training set into K equal parts (or "folds"), and then training K separate models, each with one of the K parts as validation data and the others as training data. 

Cross-validation is implemented in scikit-learn's [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function. We will use 3-fold cross-validation to evaluate the accuracy of our three models: `model_nonorm`, `model_norm`, and `pipeline`.

Note the following:
1. The first three arguments for the `cross_val_score` are the model, the training input data, and the training output data. These last two entries are the same as were passed to the `fit` function in the previous part. 
2. Use `scoring='accuracy'` to set the evaluation metric to accuracy. Use `cv=3` to set the number of folds to 3. 
3. The function should return 3 values of accuracy -- one for each of the folds. Store the *mean* of these as `acc_nonorm`, `acc_norm`, and `acc_pipe` for the un-normalized, normalized, and pipeline models respectively. 
4. Note the improvement due to normalization. What do you think might account for the difference?

In [ ]:
from sklearn.model_selection import cross_val_score

acc_nonorm = ...      #  TODO
acc_norm = ...      #  TODO
acc_pipe = ...      #  TODO

In [ ]:
grader.check("q7")

# 8. Evaluate the models with test data

We can also use the test data to evaluate the performance of our three models. Here we will use the [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) method to compute the test accuracy in each case. 

`accuracy_score` takes as inputs the true and the predicted values. Use the `predict` function of the model to obtain the predicted output. Then pass the predictions along with the true values of the output into `accuracy_score`.
Do this for the non-normalized model (`model_nonorm`) and for the pipeline model (`pipeline`). Save the results to `acc_nonorm_test` and `acc_pipe_test` respectively. How do these values compare to the accuracies obtained by cross-validation?

In [ ]:
from sklearn.metrics import accuracy_score

yhat_nonorm = model_nonorm.predict(...)
acc_nonorm_test = accuracy_score(..., ...)

yhat_pipe = pipeline.predict(...)
acc_pipe_test = accuracy_score(..., ...)

In [ ]:
grader.check("q8")


---

<h1><center><font color='purple'>Regularization</font><br></center></h1>


# 9. LASSO regularized logistic regression

Regularization is a method for avoiding overfitting by penalizing the complexity of the model in the training process. LASSO regularization in particular penalizes the sum of the absolute values of the parameters. It has the property that it will tend to "zero out" coefficients as the penalty $\lambda$ increases. This gives it an additional use as a feature selector. 

In this part we will train a LASSO regularized logistic regression model. Instead of $\lambda$, we will use the `C` parameter of `LogisticRegression`, which is the inverse of $\lambda$. 

The code iterates through a logarithmically spaced array of regularization parameters `C`. For each value it trains and evaluates a logistic regression pipeline with the regularization parameter set to that value. 

Your task is to complete the code. Your pipeline should have two componenents: a `StandardScaler` for normalizing the data, followed by a `LogisticRegression` regression model. When building the pipeline, you should pass these parameters to the `LogisticRegression` constructor: 

```python 
C=C[c],
penalty='l1',
solver='liblinear',
```

in addition to setting the random state. 

In [ ]:
C = np.logspace(-2,2,20)
acc = np.empty(20)
models = list()

for c in range(len(C)):   
    
    print(c)
    
    model = Pipeline([...
                      ... ])
    model.fit(...,...)
    
    models.append(model)

    # Validation accuracy
    acc[c] = cross_val_score(model, Xtrain, ytrain, cv=3, scoring='accuracy').mean()


In [ ]:
grader.check("q9")

# 10. Choose the best model

Next we select the model with the best validation accuracy. Follow the steps in the code. 

In [ ]:
# 1. Set `cstar` to the index of the best performing regularization value
cstar = ...

# 2. Set `acc_star` to the corresponding accuracy value
acc_star = ...

# The next bit of code extracts the coefficients of the logistic regression for each of the 20 values of `C`. 
# This is stored in `theta` , which is a (20,30) array. (30 is the number of features)
theta = np.vstack([model.named_steps['logreg'].coef_[0,:] for model in models])

# 3. Plot the validation accuracy as a function of `C`. (done already)
fig, ax = plt.subplots(figsize=(8,8),nrows=2,sharex=True)
ax[0].semilogx(C,acc,'o-',color='b',linewidth=2)
ax[0].semilogx(C[cstar],acc_star,'*',color='b',markersize=14)
ax[0].grid(linestyle=':')
ax[0].set_ylabel('validation accuracy',fontsize=12)

# 4. In a single plot, plot the 30 coefficients as a fucntion of `C`.
ax[1].semilogx(C,theta)
ax[1].grid(linestyle=':')
ax[1].set_xlabel('C',fontsize=16)

In [ ]:
grader.check("q10")

# 11. Significant features

The plot below shows the coefficients for the best-case regularized logistic regression found in the previous part. Notice that many of these coefficients have been set to zero. 

In [ ]:
theta_star = theta[cstar,:]

plt.figure(figsize=(10,3))
plt.stem(np.abs(theta_star))

In [ ]:
features = Xtrain.columns

# 1. Set `best_features` to the set of feature names corresponding to non-zero coefficients in the plot above. 
best_features = ...

# 2. Set `max_theta_feature` to the feature name corresponding to the coefficient with maximum absolute value. 
max_theta_feature = ...

# 3. Save the selected lasso model to the variable `lasso_model`.
lasso_model = ...

In [ ]:
grader.check("q11")

# 12. Evaluate the final model with test data

Use the test dataset to evaluate the accuracy of the selected LASSO model. 

In [ ]:
yhat = ...
lasso_test = ...

In [ ]:
grader.check("q12")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)